In [1]:
from proveit import specialStatementMagic # for %begin_theorems and %end_theorems
from proveit.basiclogic import In, Forall, Equals, NotEquals
from proveit.number import Summation, DiscreteContiguousSet, Abs, Neg, Add, Subtract, Multiply, Fraction, Exponentiate
from proveit.number import GreaterThan, LessThanEquals, DiscreteContiguousSet, Mod
from proveit.statistics import Prob
from proveit.common import a, b, l, t, eps, k
from proveit.number.common import zero, one, two, four, e, pi, i, Reals, Naturals, Integers
from proveit.linalg import MatrixProd
from proveit.physics.quantum import Bra, Ket
from proveit.physics.quantum.QPE.common import phase_, m_, b_, delta_, two_pow_t, two_pow_t_minus_one, alpha_l, alpha_l_sqrd
from proveit.physics.quantum.QPE.phaseEstOps import Psuccess, Pfail, ModAdd
from IPython.display import display

Convenience methods for building expressions:

In [2]:
def exp2pi_i(*exp_factors):
    return Exponentiate(e, Multiply(*((two, pi, i) + exp_factors)))

def exp2pi_i_on_two_pow_t(*exp_factors):
    return Exponentiate(e, Fraction(Multiply(*((two, pi, i) + exp_factors)), two_pow_t))

def expNeg2pi_i_on_two_pow_t(*exp_factors):
    return Exponentiate(e, Fraction(Neg(Multiply(*((two, pi, i) + exp_factors))), two_pow_t))

display(exp2pi_i(a, b))
display(exp2pi_i_on_two_pow_t(a, b))
display(expNeg2pi_i_on_two_pow_t(a, b))

e^(2 * pi * i * a * b)

e^((2 * pi * i * a * b) / 2^(t))

e^((-(2 * pi * i * a * b)) / 2^(t))

In [3]:
%begin_theorems

Take care of number domain issues:

In [4]:
two_pow_t_in_nats = In(two_pow_t, Naturals)
two_pow_t_in_nats

2^(t) in Naturals

In [5]:
two_pow_t_not_zero = NotEquals(two_pow_t, zero)
two_pow_t_not_zero

2^(t) != 0

In [6]:
modAddClosure = Forall((a, b), In(ModAdd(a, b), Integers), domain=Integers)
modAddClosure

forall_{a, b in Integers} ((a MOD_ADD b) in Integers)

In [7]:
bestInInts = In(b_, Integers)
bestInInts

b in Integers

In [8]:
deltaInReals = In(delta_, Reals)
deltaInReals

delta in Reals

Success probability as sum of individual success event probabilities:

In [9]:
success_sum = Forall(eps, Equals(Psuccess(eps), Summation(l, Prob(Equals(Abs(Subtract(m_, b_)), l)), 
                                                          DiscreteContiguousSet(Neg(eps), eps))),
                    domain=Integers)
success_sum

forall_{eps in Integers} (Psuccess(eps) = (Summation_{l=-eps}^{eps}Pr[(|(m - b)| = l)]))

Failure probability as sum of individual failure event probabilities in terms of $\alpha_l$, amplitude of $\lvert \Psi \rangle$ for a state specified relative to $b$ (the best outcome state):

In [10]:
fail_sum = Forall(eps, Equals(Pfail(eps), Add(Summation(l, alpha_l_sqrd, 
                                                        DiscreteContiguousSet(Neg(Add(two_pow_t_minus_one, one)), 
                                                                              Neg(Add(eps, one)))),
                                              Summation(l, alpha_l_sqrd, 
                                                        DiscreteContiguousSet(Add(eps, one), 
                                                                              two_pow_t_minus_one)))),
                 domain=Integers)
fail_sum

forall_{eps in Integers} (Pfail(eps) = ((Summation_{l=-(2^(t - 1) + 1)}^{-(eps + 1)}|alpha_{l}|^(2)) + (Summation_{l=eps + 1}^{2^(t - 1)}|alpha_{l}|^(2))))

Modulo addition may be converted to regular addition within $2 \pi i$ exponentiation:

In [11]:
exp2pi_i_modadd = Forall((a, b), Equals(exp2pi_i_on_two_pow_t(ModAdd(a, b)), 
                                       exp2pi_i_on_two_pow_t(Add(a, b))), domain=Integers)
exp2pi_i_modadd

forall_{a, b in Integers} (e^((2 * pi * i * (a MOD_ADD b)) / 2^(t)) = e^((2 * pi * i * (a + b)) / 2^(t)))

Direct evaluation of $\alpha_l$:

In [12]:
alpha_l_eval = Forall(l, Equals(alpha_l, 
                                Multiply(Fraction(one, two_pow_t),
                                         Summation(k, Multiply(expNeg2pi_i_on_two_pow_t(k, ModAdd(b_, l)), 
                                                          exp2pi_i(phase_, k)),
                                                   DiscreteContiguousSet(zero, two_pow_t_minus_one)))),
                     domain=Integers)
alpha_l_eval

forall_{l in Integers} (alpha_{l} = (1 / 2^(t) * (Summation_{k=0}^{2^(t - 1)}(e^((-(2 * pi * i * k * (b MOD_ADD l))) / 2^(t)) * e^(2 * pi * i * phase * k)))))

Evaluation of $\alpha_l$ after performing the geometric series summation in terms of $\delta$:

In [13]:
phaseFromBest = Equals(phase_, Add(Fraction(b_, two_pow_t), delta_))
phaseFromBest

phase = (b / 2^(t) + delta)

In [14]:
alpha_l_summed = Forall(l, Equals(alpha_l, Multiply(Fraction(one, two_pow_t), 
                                                    Fraction(Subtract(one, exp2pi_i(Subtract(Multiply(two_pow_t, delta_), l))),
                                                             Subtract(one, exp2pi_i(Subtract(Fraction(l, two_pow_t), delta_)))))),
                       domain=Integers)
alpha_l_summed

forall_{l in Integers} (alpha_{l} = (1 / 2^(t) * (1 - e^(2 * pi * i * ((2^(t) * delta) - l))) / (1 - e^(2 * pi * i * (l / 2^(t) - delta)))))

$| \alpha_l |^2$ inequality to bound the failure probability:

In [15]:
alpha_l_sqrd_ineq = Forall(l, LessThanEquals(alpha_l_sqrd, 
                                             Fraction(one, 
                                                      Multiply(four, 
                                                               Exponentiate(Subtract(l, 
                                                                                     Multiply(two_pow_t, 
                                                                                              delta_)), 
                                                                            two)))),
                          domain=Integers)
alpha_l_sqrd_ineq

forall_{l in Integers} (|alpha_{l}|^(2) <= 1 / (4 * (l - (2^(t) * delta))^(2)))

A bound on the failure probability:

In [16]:
fail_ineq = Forall(eps, LessThanEquals(Pfail(eps), Fraction(one, Multiply(two, Subtract(eps, one)))), domain=Reals)
fail_ineq

forall_{eps in Reals} (Pfail(eps) <= 1 / (2 * (eps - 1)))

In [17]:
%end_theorems

Creating theorems *.dill and *.pv_it files in the __pv_it__ directory
These theorems may be imported from proveit.physics.quantum.QPE.theorems
